## Pre-processamento do dataset para alimentação da esteira de similaridade de textos

In [1]:
import string
import nltk
from nltk.corpus import stopwords
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pandas as pd
import re
import numpy as np
import math
import random
from nltk.corpus import wordnet

[nltk_data] Downloading package wordnet to /home/douglas/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Opening movie reviews dataset

In [2]:
messages = pd.read_csv('./chatdata/movie_lines_normalized.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [3]:
messages.columns = ['msg_line', 'user_id', 'movie_id', 'msg']

In [4]:
messages.head(10)

,msg_line,user_id,movie_id,msg
0,L1045,u0,m0,They do not!
1,L1044,u2,m0,They do to!
2,L985,u0,m0,I hope so.
3,L984,u2,m0,She okay?
4,L925,u0,m0,Let's go.
5,L924,u2,m0,Wow
6,L872,u0,m0,Okay -- you're gonna need to learn how to lie.
7,L871,u2,m0,No
8,L870,u0,m0,"""""""I'm kidding. You know how sometimes you jus..."
9,L869,u0,m0,Like my fear of wearing pastels?


### Cleaning the msg_line of the conversations

In [5]:
#remove characters
def remove_char(txt):
    return re.sub('[^0-9]','', txt)

In [6]:
#leaving just the number of the index, so L872 changes to 872
messages['msg_line_clean'] = [remove_char(msg) for msg in messages['msg_line']]

In [7]:
#change the column type to number
messages['msg_line_clean'] = pd.to_numeric(messages['msg_line_clean'])

In [8]:
messages = messages.sort_values(by=['msg_line_clean'])

In [9]:
#set the column as the index
messages = messages.set_index('msg_line_clean')

In [10]:
messages.head(10)

,msg_line,user_id,movie_id,msg
msg_line_clean,,,,
49,L49,u0,m0,Did you change your hair?
50,L50,u3,m0,No.
51,L51,u0,m0,You might wanna think about it
59,L59,u9,m0,I missed you.
60,L60,u8,m0,It says here you exposed yourself to a group o...
61,L61,u9,m0,It was a bratwurst. I was eating lunch.
62,L62,u8,m0,With the teeth of your zipper?
63,L63,u7,m0,You the new guy?
64,L64,u2,m0,So they tell me...


### Removing entities of messages like PEOPLE or ORG

In [11]:
entities = pd.read_csv('./chatdata/entity_list_complete.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [12]:
entities.columns = ['ent', 'type']

In [13]:
entities['ent'] = entities['ent'].str.lower()

In [14]:
entities.head()

,ent,type
0,kinda,ORG
1,the dallas times herald,ORG
2,queen louisa,PERSON
3,a.m,GPE
4,cousin hop,PERSON


In [15]:
entities['ent_len'] = [len(e) for e in entities['ent']]

In [16]:
s = entities['ent_len'].sort_values(ascending=False).index

In [17]:
entities = entities.reindex(s)

In [18]:
entities = entities.reset_index(drop=True)

In [19]:
entities.head()

,ent,type,ent_len
0,"""""""how can the bolshevik cause gain respect am...",WORK_OF_ART,237
1,"""""""the premier wishes to inform the government...",WORK_OF_ART,192
2,""""""" come tuesday twelve a.m. bingo these like-...",WORK_OF_ART,182
3,"""""""the suggestion of the president regarding t...",WORK_OF_ART,155
4,"""""""the management of boyd's takes pleasure in ...",WORK_OF_ART,146


In [20]:
data = messages['msg']

In [21]:
ent_list =  ['PERSON', 'ORG', 'NORP', 'FAC', 'GPE', 'LOC', 'PRODUCT', 'EVENT', 'WORK_OF_ART', 'LAW', 'LANGUAGE']
#ent_list =  ['LANGUAGE']

ent = list()
for i in range(len(entities.index)):
    if entities['type'][i] in ent_list:
        ent.append(entities['ent'][i])

In [22]:
ent = list(set(ent))
print(len(ent))
print(ent)

27848
['oofh', 'fruitful yuki', 'nicholai tachlinkov', 'matthew ai', '3rd street', 'sam yeah', 'soranhas', 'brave', 'strychnine', 'stoley spritzers', 'winthrop', 'roger meris', 'the crescent moon', 'the reflecting pond', 'anthony adams', 'remember daniel grant', "all soul's eve", '"""how are you?"', 'arctic', 'trans', 'phyliss', 'marshals mortier', 'philip', 'prima', 'motley crue', 'in--', 'custard', 'taj mahal', 'curtains', 'now this i want to hear', 'd.j. johnny crunch', 'don ignacio fernandez', 'hebrew national', 'leo strauss', 'plymouth', 'disney', 'diz moore--poet', 'midlothian', 'bismark', 'kevlar second chance', 'jesus selina', 'privy councillor', 'martin swayzak', 'the neutral zone', 'baltus', 'son of sam', 'wong henry wong', 'loew', 'christopher', 'rachel jamie', 'shuttlebay', 'theo and nels', '""" research."', 'the hall of wimps', 'tor johnson', 'lock', 'quarantine', 'kingston penitentiary 1937', 'disco', 'herne', 'revelations ii', 'orinco', 'the war department', 'ted bundy',

In [23]:
dict = {}
for n in ent:
    dict[n] = len(n)

In [24]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [25]:
dict

{'"""how can the bolshevik cause gain respect among the moslems if your three representatives buljanoff iranoff and kopalski get so drunk that they throw a carpet out of their hotel window and complain to the management that it didn\'t fly"': 237,
 '"""the premier wishes to inform the government of the united states that it will be impossible for him to attend the meeting suggested by the president unless the meeting is held in moscow."""': 192,
 '""" come tuesday twelve a.m. bingo these like-minded deviates log on and start yakking it up: explicit sex crime gossip who did what to whom who wants to do what when why and how."""': 182,
 '"""the suggestion of the president regarding the possibility of a meeting in moscow would be unacceptable to her majesty\'s government at the present time."': 155,
 '"""the management of boyd\'s takes pleasure in requesting the company of mr. richard starkey that\'s you in their recently refinished gaming rooms."': 146,
 '"""well jim i says, it makes me 

In [26]:
def remove_entity(corpus):
    corpus = corpus.split(' ')
    corpus = [c for c in corpus if c not in list(dict.keys())]
    return ' '.join(corpus)    

In [27]:
%%time
messages['msg_pre_processed'] = [remove_entity(m) for m in list(data)]

CPU times: user 1h 44min 18s, sys: 11.5 s, total: 1h 44min 29s
Wall time: 1h 48min 30s


### Opening conversation sequence dataset

In [29]:
#read the file with the conversation sequence
conv_seq = pd.read_csv('./chatdata/movie_conversations.tsv', header = None, delimiter="\t", quoting=3, encoding='ISO-8859-2')

In [31]:
conv_seq.columns = ['user1_id', 'user2_id', 'movie_id', 'sequence']

In [32]:
conv_seq.head(10)

,user1_id,user2_id,movie_id,sequence
0,u0,u2,m0,['L194' 'L195' 'L196' 'L197']
1,u0,u2,m0,['L198' 'L199']
2,u0,u2,m0,['L200' 'L201' 'L202' 'L203']
3,u0,u2,m0,['L204' 'L205' 'L206']
4,u0,u2,m0,['L207' 'L208']
5,u0,u2,m0,['L271' 'L272' 'L273' 'L274' 'L275']
6,u0,u2,m0,['L276' 'L277']
7,u0,u2,m0,['L280' 'L281']
8,u0,u2,m0,['L363' 'L364']
9,u0,u2,m0,['L365' 'L366']


### Build conversation sequence

In [33]:
def split_conversation(txt):
    txt_alt = txt.split(' ')
    return txt_alt

In [34]:
def seq_to_list(seq):
    seq_list = [remove_char(s) for s in seq]
    return seq_list

In [35]:
#initializing the msg_2 column
messages['msg_2'] = '-'

In [36]:
def link_conversations(seq_list, df, filter1, filter2):
    i = 0
    while i in range(len(seq_list)):
        if i+1 < len(seq_list):
            next_msg = df.loc[int(seq_list[i+1]), filter1]
            df.at[int(seq_list[i]), filter2] = next_msg
        i+=1

In [37]:
#link each message with its answer
for conv in conv_seq['sequence']:
    #split each sequence by space
    seq = split_conversation(conv)

    #remove the char L from the sequences
    txt_alt = [remove_char(s) for s in seq]

    #use the conversation sequence to build the target answer for each message
    link_conversations(txt_alt, messages, 'msg', 'msg_2')

In [38]:
messages.head(30)

,msg_line,user_id,movie_id,msg,msg_pre_processed,msg_2
msg_line_clean,,,,,,
49,L49,u0,m0,Did you change your hair?,Did you change your hair?,No.
50,L50,u3,m0,No.,No.,You might wanna think about it
51,L51,u0,m0,You might wanna think about it,You might think about it,-
59,L59,u9,m0,I missed you.,I missed you.,It says here you exposed yourself to a group o...
60,L60,u8,m0,It says here you exposed yourself to a group o...,It says here you to a of freshmen girls.,It was a bratwurst. I was eating lunch.
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,It was a bratwurst. I was eating lunch.,With the teeth of your zipper?
62,L62,u8,m0,With the teeth of your zipper?,With the of your zipper?,-
63,L63,u7,m0,You the new guy?,You the new guy?,So they tell me...
64,L64,u2,m0,So they tell me...,So they tell me...,C'mon. I'm supposed to give you the tour.


## Pre processing the msg

In [39]:
data = messages['msg']

In [40]:
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [41]:
lemmatizer = WordNetLemmatizer()
def pre_processing_text(corpus):
    #remove html tags
    corpus = re.sub(r'<.*?>', '', str(corpus))
    
    #remove non-alphanumeric characters
    corpus = re.sub(r'[^a-z A-Z 0-9 \s]', '', str(corpus))
    
    #remove duplicated spaces
    corpus = re.sub(r' +', ' ', str(corpus))
    
    #capitalization
    corpus = corpus.lower()
    
    #tokenization
    corpus = re.findall(r"\w+(?:'\w+)?|[^\w\s]", corpus)
    
    #lammatization
    corpus = [lemmatizer.lemmatize(c, get_wordnet_pos(c)) for c in corpus]
    
    #remove punctuation
    corpus = [t for t in corpus if t not in string.punctuation]
    
    #remove stopwords
    stopwords_ = stopwords.words("english")
    corpus = [t for t in corpus if t not in stopwords_]
    
    corpus = ' '.join(corpus)

    return corpus

In [42]:
%%time
data_pre_processed = [pre_processing_text(str(m)) for m in data]
data_pre_processed

CPU times: user 11min 28s, sys: 4min 46s, total: 16min 14s
Wall time: 17min 3s


['change hair',
 '',
 'might wanna think',
 'miss',
 'say expose group freshman girl',
 'bratwurst eat lunch',
 'teeth zipper',
 'new guy',
 'tell',
 'cmon im suppose give tour',
 'dakota',
 'north actually howd',
 'kid people actually live',
 'yeah couple outnumber cow though',
 'many people old school',
 'thirtytwo',
 'get',
 'many people go',
 'couple thousand evil',
 'im use',
 'yeah guy never see horse jack clint eastwood',
 'girl',
 'burn pine perish',
 '',
 'bianca stratford sophomore dont even think',
 '',
 'could start haircut doesnt matter shes allow date old sister thats impossibility',
 'katarina stratford youve terrorize blaise',
 'express opinion terrorist action',
 'well yes compare choice expression year today event quite mild way bobby rictors gonad retrieval operation go quite well case youre interested',
 'still maintain kick ball merely spectator',
 'point kat people perceive somewhat',
 'tempestuous',
 'believe heinous bitch term use often',
 '',
 'patrick verona r

In [43]:
messages['msg_pre_processed'] = data_pre_processed

### Checking and removing duplicated messages in msg field

In [44]:
data = messages['msg_pre_processed']

In [45]:
dict = {}
for n in data:
    if n in dict:
        dict[n] = dict[n] + 1
    else:
        dict[n] = 1

In [46]:
#sort dict by biggest values
dict = {k: v for k, v in sorted(dict.items(), key=lambda item: item[1], reverse=True)}

In [47]:
dict

{'': 12843,
 'yes': 2121,
 'yeah': 1552,
 'know': 876,
 'go': 692,
 'okay': 653,
 'right': 629,
 'dont know': 629,
 'sure': 554,
 'oh': 535,
 'say': 514,
 'get': 508,
 'whats': 467,
 'mean': 462,
 'well': 441,
 'come': 411,
 'think': 406,
 'want': 393,
 'thank': 379,
 'talk': 351,
 'really': 347,
 'dont': 325,
 'like': 319,
 'hello': 315,
 'unknown': 310,
 'thanks': 305,
 'happen': 291,
 'yes sir': 289,
 'course': 277,
 'see': 276,
 'good': 263,
 'nothing': 263,
 'tell': 256,
 'huh': 253,
 'hi': 252,
 'fuck': 249,
 'im sorry': 247,
 'sir': 241,
 'cant': 237,
 'hell': 220,
 'look': 214,
 'thats right': 202,
 'uhhuh': 195,
 'love': 191,
 'hey': 187,
 'im': 177,
 'whats wrong': 173,
 'thats': 165,
 'fine': 162,
 'one': 162,
 'way': 156,
 'sorry': 156,
 'time': 151,
 'shit': 147,
 'excuse': 145,
 'shut': 145,
 'guess': 140,
 'oh god': 139,
 'please': 138,
 'dont think': 136,
 'didnt': 135,
 'stop': 135,
 'maybe': 130,
 'oh yeah': 127,
 'told': 119,
 'wait': 117,
 'ok': 116,
 'help': 115,
 

In [48]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'yes']

,msg_line,user_id,movie_id,msg,msg_pre_processed,msg_2
msg_line_clean,,,,,,
817,L817,u9,m0,Yes,yes,I don't like to do what people expect. Then th...
2571,L2571,u30,m2,Yes.,yes,Please join us. Come on forward.
2915,L2915,u30,m2,Yes. Both.,yes,-
3968,L3968,u50,m3,Yes?,yes,Well I got the papers on my official up-gradin...
3975,L3975,u50,m3,Yes.,yes,Yesterday I finally called the Accounting Offi...
...,...,...,...,...,...,...
665827,L665827,u9017,m615,Yes.,yes,You're just supposed to repeat what you said.
666075,L666075,u9020,m615,Yes I will.,yes,...But an intellectual relationship like this ...
666154,L666154,u9020,m615,Yes.,yes,What d'ya want to do to kill time?


In [49]:
#get the repeated messages
d_list = list()
for k in dict:
    if dict[k] > 1:
        d_list.append(k)

In [50]:
d_list

['',
 'yes',
 'yeah',
 'know',
 'go',
 'okay',
 'right',
 'dont know',
 'sure',
 'oh',
 'say',
 'get',
 'whats',
 'mean',
 'well',
 'come',
 'think',
 'want',
 'thank',
 'talk',
 'really',
 'dont',
 'like',
 'hello',
 'unknown',
 'thanks',
 'happen',
 'yes sir',
 'course',
 'see',
 'good',
 'nothing',
 'tell',
 'huh',
 'hi',
 'fuck',
 'im sorry',
 'sir',
 'cant',
 'hell',
 'look',
 'thats right',
 'uhhuh',
 'love',
 'hey',
 'im',
 'whats wrong',
 'thats',
 'fine',
 'one',
 'way',
 'sorry',
 'time',
 'shit',
 'excuse',
 'shut',
 'guess',
 'oh god',
 'please',
 'dont think',
 'didnt',
 'stop',
 'maybe',
 'oh yeah',
 'told',
 'wait',
 'ok',
 'help',
 'work',
 'dont understand',
 'much',
 'jesus',
 'whats matter',
 'would',
 'let go',
 'long',
 'believe',
 'alright',
 'great',
 'kill',
 'dad',
 'dont believe',
 'youre',
 'call',
 'take',
 'whats go',
 'ask',
 'bad',
 'could',
 'leave',
 'never',
 'name',
 'yet',
 'wont',
 'understand',
 'exactly',
 'forget',
 'im sure',
 'whats name',
 'ki

In [51]:
messages = messages.drop_duplicates(subset=['msg_pre_processed'])

In [52]:
#example of duplcated msg
messages[messages['msg_pre_processed'] == 'yes']

,msg_line,user_id,movie_id,msg,msg_pre_processed,msg_2
msg_line_clean,,,,,,
817,L817,u9,m0,Yes,yes,I don't like to do what people expect. Then th...


### Removing empty messages

In [53]:
messages[messages['msg_pre_processed'] == '']

,msg_line,user_id,movie_id,msg,msg_pre_processed,msg_2
msg_line_clean,,,,,,
50,L50,u3,m0,No.,,You might wanna think about it


In [54]:
messages = messages.drop(messages[messages['msg_pre_processed'] == ''].index)

### Removing nan msg origined by '' messages

In [55]:
messages[messages['msg_pre_processed'].isna()]

,msg_line,user_id,movie_id,msg,msg_pre_processed,msg_2
msg_line_clean,,,,,,


In [56]:
#filling the nan messages with a string- not necessary
#messages = messages.fillna('-')
messages = messages.dropna()

### Removing apostrophes (need for embedding and page rank  dictionary)

In [57]:
messages['msg_pre_processed'] = [ word.replace("\'","") for word in messages['msg_pre_processed'] ]

### Filling '-' or '' messages with a generic one (not used)

### Removing messages without answer

In [58]:
messages = messages.drop(messages[messages['msg_2'] == '-'].index)
messages = messages.drop(messages[messages['msg_2'] == ''].index)

### Tagging the msg with classes: 1 is question, 0 is answer

In [59]:
def define_target(corpus):
    
    if '?' in corpus:
        return 1
    else:
        return 0

In [60]:
data = messages['msg']

In [61]:
messages['target'] = [define_target(m) for m in data]

In [62]:
messages['target'] = messages['target'].astype(int)

### Save data

In [63]:
messages

,msg_line,user_id,movie_id,msg,msg_pre_processed,msg_2,target
msg_line_clean,,,,,,,
49,L49,u0,m0,Did you change your hair?,change hair,No.,1
59,L59,u9,m0,I missed you.,miss,It says here you exposed yourself to a group o...,0
60,L60,u8,m0,It says here you exposed yourself to a group o...,say expose group freshman girl,It was a bratwurst. I was eating lunch.,0
61,L61,u9,m0,It was a bratwurst. I was eating lunch.,bratwurst eat lunch,With the teeth of your zipper?,0
63,L63,u7,m0,You the new guy?,new guy,So they tell me...,1
...,...,...,...,...,...,...,...
666504,L666504,u9032,m616,Eleven o'clock My LorJ 42,eleven oclock lorj 42,Our friend Colonel Dumford will be should be a...,0
666520,L666520,u9034,m616,Well I assure you Sir I have no desire to crea...,well assure sir desire create difficulty 45,And I assure you you do not In fact I'd be obl...,0
666521,L666521,u9030,m616,And I assure you you do not In fact I'd be obl...,assure fact id oblige best advice scout see,So far only their scouts. But we have had repo...,1


In [64]:
messages.to_csv('./chatdata/movie_lines_pre_processed_similarity.tsv', index=False, sep='\t', header=False)

In [65]:
# saving a small dataset pre processed
messages[0:30000].to_csv('./chatdata/movie_lines_pre_processed_for_test.tsv', index=False, sep='\t', header=False)